### 原理
#### 核心机制
1. **哈希表存储**：使用字典存储函数参数和返回值的映射关系  
2. **参数作为键**：将函数的所有参数转换为可哈希的键  
3. **结果缓存**：相同的参数调用直接返回缓存结果，避免重复计算

底层实现

In [ ]:
# 简化的cache实现原理
def cache(user_function):
    cache_dict = {}

    def wrapper(*args, **kwargs):
        # 创建缓存键
        key = (args, tuple(kwargs.items()))

        if key not in cache_dict:
            # 缓存未命中，执行函数并缓存结果
            cache_dict[key] = user_function(*args, **kwargs)

        # 返回缓存结果
        return cache_dict[key]

    return wrapper

#### 使用方式
基本使用

In [ ]:
from functools import cache


@cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


# 第一次计算会执行函数
print(fibonacci(10))  # 55
# 第二次相同参数直接返回缓存结果
print(fibonacci(10))  # 立即返回55

带参数的函数

In [ ]:
@cache
def expensive_computation(a, b, c=10):
    print(f"计算 {a}, {b}, {c}")  # 只会打印一次
    return a * b * c


result1 = expensive_computation(3, 4, c=5)  # 会打印
result2 = expensive_computation(3, 4, c=5)  # 不会打印，使用缓存

与 `@lru_cache` 的区别
|特性|	@cache	|@lru_cache|
|:-:|:-:|:-:|
|缓存大小|	无限制|	可限制大小|
|内存管理	|可能内存泄漏|	LRU淘汰策略|
|Python版本|	3.9+|	3.2+|

In [ ]:
from functools import lru_cache


# @cache 等价于 @lru_cache(maxsize=None)
@cache  # 无限制缓存
def func1():
    pass


@lru_cache(maxsize=None)  # 同上
def func2():
    pass


@lru_cache(maxsize=128)  # 限制缓存大小
def func3():
    pass

#### 实际应用场景
1. 递归优化


In [ ]:
@cache
def combinations(n, k):
    if k == 0 or k == n:
        return 1
    return combinations(n - 1, k - 1) + combinations(n - 1, k)


print(combinations(50, 25))  # 大量重复计算被优化

2. API调用缓存

In [ ]:
import requests
from functools import cache


@cache
def get_weather(city):
    print(f"调用API获取{city}天气")
    # 模拟API调用
    time.sleep(2)
    return f"{city}: 晴朗"


# 第一次调用实际请求
weather1 = get_weather("北京")  # 打印并等待
# 后续相同参数使用缓存
weather2 = get_weather("北京")  # 立即返回

3. 配置加载

In [ ]:
@cache
def load_config(config_path):
    print(f"加载配置文件: {config_path}")
    # 模拟读取文件
    with open(config_path, 'r') as f:
        return f.read()


# 多次调用只加载一次
config1 = load_config("config.json")
config2 = load_config("config.json")  # 使用缓存

#### 注意事项  
1. 参数必须可哈希

In [ ]:
@cache
def process_data(data):
    return sum(data)


# 错误：列表不可哈希
# process_data([1, 2, 3])

# 正确：使用元组
process_data((1, 2, 3))

2. 副作用函数慎用

In [ ]:
counter = 0


@cache
def with_side_effect():
    global counter
    counter += 1
    return counter


print(with_side_effect())  # 1
print(with_side_effect())  # 1（不是2！）

3. 内存使用监控

In [ ]:

from functools import cache


@cache
def large_memory_function(n):
    return [i for i in range(n)]


# 缓存大量结果可能导致内存问题
results = []
for i in range(1000):
    results.append(large_memory_function(10000))

#### 高级技巧  
1. 手动缓存控制

In [ ]:
from functools import cache


@cache
def cached_func(x):
    return x * x


# 查看缓存信息
print(cached_func.cache_info())

# 清空缓存
cached_func.cache_clear()

2. 类方法缓存  
类方法缓存指的是在类的方法上使用缓存装饰器，包括：
+ 实例方法缓存：缓存基于实例和参数
+ 类方法缓存：缓存基于类和参数
+ 静态方法缓存：缓存与实例无关

实例方法缓存

In [ ]:
from functools import cache


class Calculator:
    def __init__(self, base):
        self.base = base

    @cache
    def power(self, n):
        """计算 base 的 n 次方，结果会被缓存"""
        print(f"计算 {self.base}^{n}")
        time.sleep(1)  # 模拟复杂计算
        return self.base ** n


# 使用示例
calc1 = Calculator(2)
calc2 = Calculator(3)

# 相同实例的相同参数会使用缓存
print(calc1.power(3))  # 计算 2^3，等待1秒
print(calc1.power(3))  # 立即返回，使用缓存

# 不同实例不会共享缓存（即使参数相同）
print(calc2.power(3))  # 计算 3^3，等待1秒
print(calc2.power(3))  # 立即返回，使用缓存

类方法缓存(使用 `@classmethod`)

In [ ]:
from functools import cache


class Database:
    _connection_count = 0

    @classmethod
    @cache
    def get_connection_pool(cls, pool_size):
        """获取数据库连接池，相同的pool_size会复用"""
        print(f"{cls.__name__}: 创建 {pool_size} 个连接的连接池")
        cls._connection_count += 1
        return f"连接池-{pool_size}-{cls._connection_count}"


# 相同类、相同参数会使用缓存
print(Database.get_connection_pool(10))  # 创建
print(Database.get_connection_pool(10))  # 使用缓存
print(Database.get_connection_pool(20))  # 创建新的


# 子类继承时
class MySQLDatabase(Database):
    pass


print(MySQLDatabase.get_connection_pool(10))  # 会重新创建，因为cls不同

静态方法缓存


In [ ]:
from functools import cache


class StringUtils:
    @staticmethod
    @cache
    def levenshtein_distance(s1, s2):
        """计算编辑距离，结果会被缓存"""
        print(f"计算 '{s1}' 和 '{s2}' 的编辑距离")
        # 简化的编辑距离计算
        if len(s1) == 0:
            return len(s2)
        if len(s2) == 0:
            return len(s1)

        if s1[0] == s2[0]:
            return StringUtils.levenshtein_distance(s1[1:], s2[1:])

        return 1 + min(
            StringUtils.levenshtein_distance(s1[1:], s2),  # 删除
            StringUtils.levenshtein_distance(s1, s2[1:]),  # 插入
            StringUtils.levenshtein_distance(s1[1:], s2[1:])  # 替换
        )


# 使用示例
print(StringUtils.levenshtein_distance("kitten", "sitting"))
print(StringUtils.levenshtein_distance("kitten", "sitting"))  # 使用缓存

类方法缓存的注意事项

In [ ]:
from functools import cache


class User:
    def __init__(self, user_id, name):
        self.user_id = user_id
        self.name = name

    def __hash__(self):
        """必须实现 __hash__ 方法才能在缓存中使用对象作为键的一部分"""
        return hash(self.user_id)

    def __eq__(self, other):
        """必须实现 __eq__ 方法用于比较"""
        return isinstance(other, User) and self.user_id == other.user_id

    @cache
    def get_user_info(self, include_details=False):
        """获取用户信息，结果会被缓存"""
        print(f"获取用户 {self.name} 的信息")
        return {
            "id": self.user_id,
            "name": self.name,
            "details": "详细信息" if include_details else None
        }


# 现在可以使用缓存了
user = User(1, "张三")
print(user.get_user_info(True))
print(user.get_user_info(True))  # 使用缓存

3. 条件缓存  
条件缓存是指根据特定条件动态决定是否使用缓存。这在以下场景很有用：
+ 开发环境禁用缓存，生产环境启用缓存
+ 根据参数值决定是否缓存
+ 根据配置开关控制缓存

基本条件缓存装饰器

In [ ]:
def conditional_cache(condition=True):
    """根据条件启用或禁用缓存的装饰器"""

    def decorator(func):
        if condition:
            # 启用缓存
            cached_func = cache(func)

            @wraps(func)
            def wrapper(*args, **kwargs):
                return cached_func(*args, **kwargs)

            # 复制缓存相关的方法
            wrapper.cache_info = cached_func.cache_info
            wrapper.cache_clear = cached_func.cache_clear

            return wrapper
        else:
            # 禁用缓存，直接返回原函数
            return func

    return decorator


# 使用示例
@conditional_cache(condition=True)  # 启用缓存
def cached_function(x):
    print(f"计算 {x}")
    return x * x


@conditional_cache(condition=False)  # 禁用缓存
def uncached_function(x):
    print(f"计算 {x}")
    return x * x


print(cached_function(5))  # 打印并计算
print(cached_function(5))  # 不打印，使用缓存
print(uncached_function(5))  # 打印
print(uncached_function(5))  # 再次打印

基于环境的条件缓存

In [ ]:

import os


def environment_based_cache():
    """根据环境变量决定是否启用缓存"""

    def decorator(func):
        # 检查环境变量
        if os.getenv("ENABLE_CACHE", "false").lower() == "true":
            # 生产环境：启用缓存
            cached_func = cache(func)

            @wraps(func)
            def wrapper(*args, **kwargs):
                return cached_func(*args, **kwargs)

            wrapper.cache_info = cached_func.cache_info
            wrapper.cache_clear = cached_func.cache_clear

            return wrapper
        else:
            # 开发环境：禁用缓存
            return func

    return decorator


# 使用示例
@environment_based_cache()
def api_call(endpoint):
    print(f"调用API: {endpoint}")
    # 模拟API调用
    return f"来自 {endpoint} 的响应"

# 根据环境变量 ENABLE_CACHE 的值决定是否缓存

参数依赖的条件缓存


In [ ]:
from functools import cache


def cache_if(should_cache_func):
    """根据参数值决定是否缓存"""

    def decorator(func):
        cached_func = cache(func)

        @wraps(func)
        def wrapper(*args, **kwargs):
            # 调用 should_cache_func 判断是否应该缓存
            if should_cache_func(*args, **kwargs):
                # 使用缓存版本
                return cached_func(*args, **kwargs)
            else:
                # 不使用缓存，直接调用原函数
                return func(*args, **kwargs)

        # 提供访问缓存信息的方法
        wrapper.cache_info = cached_func.cache_info
        wrapper.cache_clear = cached_func.cache_clear

        return wrapper

    return decorator


# 使用示例：只缓存偶数结果
def should_cache_even_only(x):
    """只缓存偶数参数的结果"""
    return x % 2 == 0


@cache_if(should_cache_even_only)
def expensive_operation(x):
    print(f"计算 f({x})")
    return x ** 3


print(expensive_operation(2))  # 计算并缓存
print(expensive_operation(2))  # 使用缓存
print(expensive_operation(3))  # 计算，但不缓存
print(expensive_operation(3))  # 再次计算

带TTL的条件缓存


In [ ]:
from functools import wraps
import time


def cache_with_ttl(ttl_seconds):
    """带过期时间的缓存装饰器"""

    def decorator(func):
        cache_data = {}

        @wraps(func)
        def wrapper(*args, **kwargs):
            key = (args, tuple(kwargs.items()))

            # 检查缓存中是否有数据且未过期
            if key in cache_data:
                result, timestamp = cache_data[key]
                if time.time() - timestamp < ttl_seconds:
                    return result

            # 缓存未命中或已过期，执行函数
            result = func(*args, **kwargs)
            cache_data[key] = (result, time.time())

            # 清理过期缓存（可选）
            current_time = time.time()
            expired_keys = [
                k for k, (_, ts) in cache_data.items()
                if current_time - ts >= ttl_seconds
            ]
            for k in expired_keys:
                del cache_data[k]

            return result

        # 添加缓存管理方法
        wrapper.cache_clear = lambda: cache_data.clear()
        wrapper.cache_info = lambda: f"缓存项数: {len(cache_data)}"

        return wrapper

    return decorator


# 使用示例
@cache_with_ttl(ttl_seconds=5)  # 缓存5秒
def get_current_weather(city):
    print(f"获取 {city} 的天气")
    return f"{city}: {time.time()}"


print(get_current_weather("北京"))  # 获取
time.sleep(3)
print(get_current_weather("北京"))  # 使用缓存
time.sleep(3)
print(get_current_weather("北京"))  # 重新获取（已过期）

### 总结
`@cache` 是一个简单但强大的性能优化工具，特别适用于：  
+ 纯函数（相同输入总是相同输出）  
+ 计算密集型函数
+ 递归函数优化
+ 重复调用相同参数的函数

使用时应考虑：  
+ 函数参数是否可哈希
+ 缓存是否会导致内存问题
+ 函数是否有副作用

对于需要限制内存使用的场景，建议使用 `@lru_cache(maxsize=...)` 替代。